<a href="https://colab.research.google.com/github/Ishitty/Ishitty/blob/main/Text_To_Image_Ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: create a python code for making ai text to image generator and make sure to use gui for gradio make code long

# First, install the necessary libraries
!pip install diffusers transformers accelerate scipy ftfy gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [3]:
import torch
from diffusers import StableDiffusionPipeline
import gradio as gr
from PIL import Image

In [4]:
# Define the model ID
model_id = "runwayml/stable-diffusion-v1-5"

In [5]:
# Load the stable diffusion pipeline
# We specify torch_dtype to use float16 for memory efficiency
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
# Move the pipeline to the GPU if available
if torch.cuda.is_available():
    pipe = pipe.to("cuda")
    print("Using CUDA")
elif torch.backends.mps.is_available():
    pipe = pipe.to("mps")
    print("Using MPS (Apple Silicon)")
else:
    print("Using CPU - generation may be slow")

Using CUDA


In [7]:
# Define the image generation function
def generate_image(prompt, negative_prompt="", num_inference_steps=50, guidance_scale=7.5, num_images_per_prompt=1):
    """
    Generates an image based on the provided prompt.

    Args:
        prompt (str): The positive prompt for image generation.
        negative_prompt (str): The negative prompt (things to avoid).
        num_inference_steps (int): The number of diffusion steps. Higher values can improve quality but take longer.
        guidance_scale (float): Higher values steer the generation more towards the prompt.
        num_images_per_prompt (int): The number of images to generate for each prompt.

    Returns:
        list: A list of PIL Image objects.
    """
    if not prompt:
        return [Image.new('RGB', (512, 512), color = 'white')] # Return a blank image if prompt is empty

    print(f"Generating image for prompt: {prompt}")
    print(f"Negative prompt: {negative_prompt}")
    print(f"Inference steps: {num_inference_steps}")
    print(f"Guidance scale: {guidance_scale}")
    print(f"Images per prompt: {num_images_per_prompt}")

    try:
        # Generate images
        with torch.no_grad(): # Disable gradient calculation for faster inference
            images = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
                num_images_per_prompt=num_images_per_prompt
            ).images

        print("Image generation successful.")
        return images

    except Exception as e:
        print(f"An error occurred during image generation: {e}")
        # Return a blank image or an error image in case of failure
        error_image = Image.new('RGB', (512, 512), color = 'red')
        return [error_image] * num_images_per_prompt # Return multiple error images if requested

In [8]:
# Create the Gradio interface
with gr.Blocks(css="#output-image {max-width: 768px !important; height: auto !important;} #gallery {min-height: 512px !important;}") as demo:
    gr.Markdown(
        """
        # AI Text-to-Image Generator
        Enter a text prompt below and click "Generate" to create an image using Stable Diffusion.
        """
    )

    with gr.Row():
        with gr.Column(scale=2):
            prompt_input = gr.Textbox(label="Enter your prompt here", lines=2, placeholder="e.g., A photograph of an astronaut riding a horse on the moon")
            negative_prompt_input = gr.Textbox(label="Negative prompt (optional)", lines=1, placeholder="e.g., blurry, low quality, distortion")
            with gr.Accordion("Advanced Options", open=False):
                num_inference_steps_slider = gr.Slider(minimum=1, maximum=100, value=50, step=1, label="Inference Steps", info="Higher values can improve quality but take longer.")
                guidance_scale_slider = gr.Slider(minimum=1.0, maximum=20.0, value=7.5, step=0.5, label="Guidance Scale", info="Higher values steer the generation more towards the prompt.")
                num_images_slider = gr.Slider(minimum=1, maximum=4, value=1, step=1, label="Number of Images to Generate", info="Generate multiple images from the same prompt.")

            generate_button = gr.Button("Generate Image")

        with gr.Column(scale=1):
            output_gallery = gr.Gallery(label="Generated Images", show_label=True, elem_id="gallery", columns=2, object_fit="contain", height=512)


In [11]:
# Link the button click event to the generation function
with gr.Blocks(css="#output-image {max-width: 768px !important; height: auto !important;} #gallery {min-height: 512px !important;}") as demo:
    gr.Markdown(
        """
        # AI Text-to-Image Generator
        Enter a text prompt below and click "Generate" to create an image using Stable Diffusion.
        """
    )

    with gr.Row():
        with gr.Column(scale=2):
            prompt_input = gr.Textbox(label="Enter your prompt here", lines=2, placeholder="e.g., A photograph of an astronaut riding a horse on the moon")
            negative_prompt_input = gr.Textbox(label="Negative prompt (optional)", lines=1, placeholder="e.g., blurry, low quality, distortion")
            with gr.Accordion("Advanced Options", open=False):
                num_inference_steps_slider = gr.Slider(minimum=1, maximum=100, value=50, step=1, label="Inference Steps", info="Higher values can improve quality but take longer.")
                guidance_scale_slider = gr.Slider(minimum=1.0, maximum=20.0, value=7.5, step=0.5, label="Guidance Scale", info="Higher values steer the generation more towards the prompt.")
                num_images_slider = gr.Slider(minimum=1, maximum=4, value=1, step=1, label="Number of Images to Generate", info="Generate multiple images from the same prompt.")

            generate_button = gr.Button("Generate Image")

        with gr.Column(scale=1):
            output_gallery = gr.Gallery(label="Generated Images", show_label=True, elem_id="gallery", columns=2, object_fit="contain", height=512)

    generate_button.click(
        fn=generate_image,
        inputs=[
            prompt_input,
            negative_prompt_input,
            num_inference_steps_slider,
            guidance_scale_slider,
            num_images_slider
        ],
        outputs=output_gallery
    )

gr.Markdown(
    """
    **Note:** This application uses the Stable Diffusion v1.5 model. Image generation can take some time depending on the complexity of the prompt and the hardware available.
    """
)

In [ ]:
# Launch the Gradio interface
if __name__ == "__main__":
    # In Google Colab, you can use share=True to get a public URL
    # or disable share=True for a local interface
    demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://da946494d161df8df6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Generating image for prompt: moon 
Negative prompt: 
Inference steps: 50
Guidance scale: 7.5
Images per prompt: 1


  0%|          | 0/50 [00:00<?, ?it/s]

Image generation successful.
